In [51]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ue20cs344-nam-assignment-2/edgelist.csv
/kaggle/input/ue20cs344-nam-assignment-2/sample_submission.csv
/kaggle/input/ue20cs344-nam-assignment-2/node_data.csv
/kaggle/input/ue20cs344-nam-assignment-2/features.pt


In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install --upgrade torch-cluster
!pip install gensim

In [ ]:
pip install -U gensim

In [ ]:

!python -m pip install -U gensim

In [ ]:
!pip install torch-geometric

In [ ]:
import networkx as nx 
import torch
import pandas as pd
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn as nn

In [ ]:
df_nodes=pd.read_csv("/kaggle/input/ue20cs344-nam-assignment-2/node_data.csv")
df_edges=pd.read_csv("/kaggle/input/ue20cs344-nam-assignment-2/edgelist.csv")

In [ ]:
df_nodes.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

ENCODING CATEGORICAL TO NUMERICAL

In [ ]:
labels = LabelEncoder()
df_nodes['branch']=labels.fit_transform(df_nodes['branch'])

In [ ]:
df_nodes.head()

Trained without embeddings and used the file given

In [ ]:
features =torch.load("/kaggle/input/ue20cs344-nam-assignment-2/features.pt")

In [ ]:
G= nx.from_pandas_edgelist(df_edges, 'source','target')

In [ ]:
edge_index = torch.tensor(list(G.edges)).t().contiguous()
x=features
y=torch.tensor(df_nodes['branch'].values,dtype=torch.long)
data = Data(x=x,edge_index=edge_index, y=y)
data.train_mask =df_nodes['train_mask'].values
data.test_mask = df_nodes['test_mask'].values

In [ ]:
class GCNPool(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCNPool, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.pool = TopKPooling(hidden_channels, ratio=0.5)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        
    def forward(self, x, edge_index, batch):
        x = F.dropout(x, training=self.training)
        x = F.relu(self.conv1(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool(x, edge_index, None, batch)
        x = F.dropout(x, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        return F.log_softmax(x, dim=1)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(data.num_features,16,3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.NLLLoss()

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x.to(device), data.edge_index.to(device))
    loss = criterion(out[data.train_mask], data.y[data.train_mask].to(device))
    loss.backward()
    optimizer.step()
    return loss

for epoch in range(200):
    loss = train()
    print('Epoch: {}, Loss: {}'.format(epoch, loss))

In [ ]:
model.eval()
_, pred = model(data.x.to(device), data.edge_index.to(device)).max(dim=1)
test_preds = pred[data.test_mask].detach().cpu().numpy()
test_ids = df_nodes['id'][data.test_mask].values

In [ ]:
test_preds = labels.inverse_transform(test_preds)

In [ ]:
submission_df = pd.DataFrame({'id': test_ids, 'branch': test_preds})
submission_df.to_csv('submission.csv', index=False)
len(submission_df)

In [ ]:
TRAINING WITH DEEPWALK EMBEDDINGS